In [2]:
from hypster import hypster

@hypster
def test(hp):
    a = hp.select(["a", "b", "c"], default="a")

In [3]:
res = test()

{'a': 'a'}

In [4]:
%%writefile configs.py
from hypster import Select, prep
from dataclasses import dataclass

class CacheInterface:
    pass

class DiskCache(CacheInterface):
    def __init__(self, path, cache_op: str):
        self.path = path
        self.cache_op = cache_op

class MemoryCache(CacheInterface):
    def __init__(self, max_size, cache_op: str):
        self.max_size = max_size
        self.cache_op = cache_op

class SqlCache(CacheInterface):
    def __init__(self, conn_str, table):
        self.conn_str = conn_str
        self.table = table

@dataclass
class CacheManager:
    cache: CacheInterface

cache_manager = prep(CacheManager(cache=Select("cache"))) #this can also be None


cache_op = "all"

max_size = 1000
cache__mem = prep(MemoryCache(max_size=max_size, cache_op=cache_op))

path = "data/cache"
cache__disk = prep(DiskCache(path=path, cache_op=cache_op))

cache__new = prep(SqlCache(conn_str="sqlite:///data/cache.db", 
                           table="cache"))

class OpenAiDriver:
    def __init__(self, model):
        self.model = model

class AnthropicDriver:
    def __init__(self, model):
        self.model = model

llm_driver = Select("llm_driver")
llm_driver__openai = prep(OpenAiDriver(model="gpt3.5"))
llm_driver__anthropic = prep(AnthropicDriver(model="claude3.5"))

Writing configs.py


In [10]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
%autoreload 2

In [17]:
from hypster import Builder
import configs

builder = Builder().with_modules(configs)
driver = builder.build()


In [18]:
# Visualize configuration tree
from hypster import visualize_config_tree
print("\nConfiguration tree:")
print(visualize_config_tree(driver.root))


Configuration tree:
└── root (root)
    ├── cache_manager (CacheManager)
    │   └── cache (Select): cache
    │       ├── mem (MemoryCache)
    │       │   ├── max_size (reference): 1000
    │       │   └── cache_op (reference): all
    │       ├── disk (DiskCache)
    │       │   ├── path (reference): data/cache
    │       │   └── cache_op (reference) [SHARED]
    │       └── new (SqlCache)
    │           ├── conn_str (value): sqlite:///data/cache.db
    │           └── table (value): cache
    └── llm_driver (Select): llm_driver
        ├── openai (OpenAiDriver)
        │   └── model (value): gpt3.5
        └── anthropic (AnthropicDriver)
            └── model (value) [SHARED]



In [34]:
final_vars = ["cache_manager", "llm_driver"]
selections = {"llm_driver": "anthropic", 
              "cache_manager.cache": "disk"}
overrides = {"llm_driver.anthropic.model": "claude3-opus"}
results = driver.instantiate(final_vars, selections, overrides)

In [35]:
results["cache_manager"].cache.path

'data/cache'

In [36]:
# Visualize configuration tree
from hypster import visualize_config_tree
print("\nConfiguration tree:")
print(visualize_config_tree(driver.root))


Configuration tree:
└── root (root)
    ├── cache_manager (CacheManager)
    │   └── cache (Select): cache
    │       └── disk (DiskCache)
    │           ├── path (reference): data/cache
    │           └── cache_op (reference): all
    ├── llm_driver (Select): llm_driver
    │   ├── openai (OpenAiDriver)
    │   │   └── model (value): gpt3.5
    │   └── anthropic (AnthropicDriver)
    │       └── model (value) [SHARED]
    ├── mem (MemoryCache)
    │   ├── max_size (reference): 1000
    │   └── cache_op (reference) [SHARED]
    ├── disk (DiskCache) [SHARED]
    └── new (SqlCache)
        ├── conn_str (value): sqlite:///data/cache.db
        └── table (value): cache

